In [1]:
%%time
import os
import json
import pandas as pd
from datetime import datetime
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.progettoDM # Get the "twitter" database
trenitalia = db.trenitalia # Get the "tweets" collection from the "twitter" database

#staz = pd.read_csv("stazioni_final.csv", sep=";")



CPU times: user 381 ms, sys: 161 ms, total: 542 ms
Wall time: 860 ms


### Creazione campo numero_fermate

In [72]:
# creazione numero_fermate
from bson.son import SON
import pprint
pipeline = [
            {"$project":
                {
                    "numero_fermate": { "$cond": { "if": { "$isArray": "$fermate" },
                                                  "then": { "$size": "$fermate" }, "else": "NA"} }
                }}
        ]
lista = list(db.trenitalia.aggregate(pipeline))


for i in lista:
    idx = i["_id"]
    numero_fermate = i["numero_fermate"]
    db.trenitalia.find_one_and_update({"_id": idx}, 
                                 {"$set": {"numero_fermate": numero_fermate}})
    

In [159]:
# creazione durata teorica

from bson.son import SON
import pprint
pipeline = [
        {"$match": {"$or":[{"numero_fermate": {"$ne":"NA"}}, {"numero_fermate": {"$gt":0}}]}},
        {"$project": 
          {
             "partenza": { "$arrayElemAt": [ "$fermate", 0 ] },
             "arrivo": { "$arrayElemAt": [ "$fermate", -1 ] }
          }
        },
        #{"$convert": { 'input': '$dateDifference', 'to': 'int' }}
        #{"$project": {"durata": { "$toInt": "$dateDifference"}}}
        #{"$project": {"durata": {"$divide": [ "$durata", 60000 ] }}}
        {"$group": {"_id": {"partenza":"$partenza.regione","arrivo":"$arrivo.regione"},
                    "somma": {"$sum": 1}}}, 
          #          "numero_treni": {"$sum":1}}},
         #{"$sort": SON([("media_ritardo", -1), ("_id", -1)])}
        ]
lista = list(db.trenitalia.aggregate(pipeline))


In [164]:
with open('/Users/alessandro/Desktop/traffico_treni.json', 'w') as fout:
    json.dump(lista , fout)

In [41]:
from bson.son import SON
import pprint
start = datetime(2019, 1, 30, 0, 0, 0)
pipeline = [
        {"$match": {"categoria":"ES*"}},
        #{"$project": {"_id":1}},
         {"$group": {"_id": "$null","count": {"$sum": 1}}},
         #{"$sort": SON([("count", -1), ("_id", -1)])}
]
pprint.pprint(list(db.trenitalia.aggregate(pipeline)))

[{'_id': None, 'count': 8583}]


In [ ]:
from bson.son import SON
import pprint
pipeline = [
        {"$unwind":"$fermate"},
         {"$match": { "fermate.tipo_fermata": "A"}},
        {"$match":{"$expr": {"$ne": ["$fermate.ritardo", "$ritardo_finale"]}}},
         #{"$project": {"_id":1,"fermate.ritardo":1}},
        #{"$group": {"_id": "null", "count": {"$sum": 1}}},
         #{"$sort": SON([("count", -1), ("_id", -1)])}
        ]
pprint.pprint(list(db.trenitalia.aggregate(pipeline)))
#"$match":{"$expr": {"$eq": ["$a", "$b"]}}}

In [ ]:
from bson.son import SON
import pprint
pipeline = [
        {"$unwind":"$fermate"},
        #{"$match": { "fermate.tipo_fermata": "A"}},
         #{"$match": { "codice_treno": "REG5735","stato":"parzialmente soppresso"}},
        #{"$match":{"$expr": {"$ne": ["$fermate.ritardo", "$ritardo_finale"]}}},
         #{"$project": {"_id":1,"fermate.ritardo":1}},
        {"$group": {"_id": "$fermate.provincia", "count": {"$sum": 1}, "avg": {"$avg":"$fermate.ritardo"}, "index": {"$sum":1}}},
         {"$sort": SON([("count", -1), ("avg", -1)])},
        ]
lista2 = (list(db.trenitalia.aggregate(pipeline)))
for i in range(0,len(lista2)):
    if lista2[i]["avg"] is not None:
        lista2[i]["index"] = (lista2[i]["avg"] / lista2[i]["count"])*10000
with open('index.json', 'w') as fout:
    json.dump(lista2 , fout)

In [ ]:
#for j in range(0,len(lista2)):
from bson.son import SON
import pprint
pipeline = [
         {"$match": {"_id":__id}},
         {"$project": {"_id":1,"fermate.ritardo":1}},
        {"$group": {"_id": "null", "count": {"$sum": 1}}},
         {"$sort": SON([("count", -1), ("_id", -1)])}
        ]
pprint.pprint(list(db.trenitalia.aggregate(pipeline)))



In [ ]:

from bson.son import SON
import pprint
pipeline = [
         {"$unwind":"$fermate"},
         #{"$match": { "categoria":"ES*"}},
         #{"$project": {"_id":1,"codice_treno":1,"data":1,"fermate.ritardo":1}},
         {"$group": {"_id":{"_id": "$_id","regione": "$fermate.regione"}}},
         {"$group": {"_id":"$_id.regione", "count": {"$sum": 1}}},
        #{"$project": {"codice_treno":1,"fermate.ritardo":1}},
         {"$sort": SON([("count", -1), ("_id", -1)])}
        ]
#pprint.pprint(list(db.trenitalia.aggregate(pipeline)))
with open('treni_regione.json', 'w') as fout:
    json.dump(list(db.trenitalia.aggregate(pipeline)) , fout)

In [ ]:
from bson.son import SON
import pprint
pipeline = [
        {"$unwind": "$fermate"},
        {"$match": {"categoria":"ES*"}},
        #{"$project": {"_id":-1,"fermate.orario":1,"fermate.ritardo":1,
         #             "fermate.regione":1,"categoria":1}},
        #{"$group": {"_id": "$fermate.nome_stazione", "count_es": {"$sum": 1}}},
        #{"$sort": SON([("count_es", -1), ("_id", -1)])}
        ]
#pprint.pprint(list(db.trenitalia.aggregate(pipeline)))

lista = (list(db.trenitalia.aggregate(pipeline)))

In [ ]:
from bson.son import SON
import pprint
pipeline = [
        {"$unwind": "$fermate"},
        #{"$match": {"fermate.tipo_fermata": "P"}},
        #{"$project": {"_id":1}},
         {"$group": {"_id": "$fermate.provincia", "sum": {"$avg": "$fermate.ritardo"}}},
         {"$sort": SON([("avg", -1), ("count", -1)])},
        #{"$match": {"count": {"$gt":10}}},
        ]

with open('ritardi_provincia.json', 'w') as fout:
    json.dump(list(db.trenitalia.aggregate(pipeline)) , fout)

In [ ]:
from bson.son import SON
import pprint
start1 = datetime(2019, 1, 7, 0, 0, 0)
pipeline = [
        {"$unwind": "$fermate"},
        {"$match": {"fermate.regione":"LOMBARDIA","categoria":"REG"}},
        {"$project": {"hour": { "$hour": "$fermate.orario" }, "fermate.regione":1,
                    "fermate.ritardo":1}},
        {"$match": {"hour": 4}},
        {"$group": {"_id": "$fermate.ritardo", "count": {"$sum": 1}}},
        {"$sort": SON([("count", -1), ("_id", -1)])},
        
]
lista = pprint.pprint(list(db.trenitalia.aggregate(pipeline)))


### creazione dataset per la terza infografica

In [100]:
from bson.son import SON
import pprint
import pandas as pd
import numpy as np
import scipy.stats


def median(data):
    a = 1.0 * np.array(data)
    n = len(a)
    m = np.median(a)
    return m

start1 = datetime(2019, 1, 12, 0, 0, 0)
start2 = datetime(2019, 1, 19, 0, 0, 0)
start3 = datetime(2019, 1, 26, 0, 0, 0)
#start4 = datetime(2019, 1, 31, 0, 0, 0)
start5 = datetime(2019, 1, 5, 0, 0, 0)

#df = pd.DataFrame(columns=["giorno","regione","categoria","mediana","ora"])

categoria = ["REG","ES*","MET","IC","EC","EN"]

regioni_italiane = [
    'CALABRIA',
    'LIGURIA',
    'SICILIA',
    'TRENTINO ALTO ADIGE',
    'EMILIA ROMAGNA',
    'UMBRIA',
    'MARCHE',
    'ABRUZZO',
    'PIEMONTE',
    'VENETO',
    'FRIULI VENEZIA GIULIA',
    'PUGLIA',
    'TOSCANA',
    'BASILICATA',
    'CAMPANIA',
    'ESTERO',
    'SARDEGNA',
    'MOLISE',
    "VALLE D'AOSTA",
    'LAZIO',
    'LOMBARDIA'
]

    
pipeline = [
        {"$unwind": "$fermate"},
        {"$match": {"$or":[{"data":start1}, {"data":start2}, {"data":start3}, 
                           {"data":start5}]}},
        {"$project": {"hour": { "$hour": "$fermate.orario" }, "fermate.regione":1,
                      "categoria":1,"fermate.ritardo":1}},
        ]

lista = (list(db.trenitalia.aggregate(pipeline)))

for ora in range(0,24):
    L = list()
    for i in range(0,len(lista)):
        if lista[i]["hour"] == ora:
            L.append(lista[i])
        #L = [x for x in L if x is not None]
    
    for cat in categoria:
        L_cat = list()
        for j in range(0,len(L)):
            if L[j]["categoria"] == cat:
                L_cat.append(L[j])
                
        for reg in regioni_italiane:
            L2 = list()
            for k in range(0,len(L_cat)):
                if L_cat[k]["fermate"]["regione"] == reg:
                    L2.append(L_cat[k]["fermate"]["ritardo"])
            L2 = [x for x in L2 if x is not None]
            #media, m_min, m_sup = mean_confidence_interval(L2, confidence=0.95)
            mediana = median(L2)
            df = df.append({'giorno': "sabato", "regione": reg, "mediana":mediana,
                            "ora":ora,"categoria":cat}, ignore_index=True)

df = df.dropna()

In [112]:
df.to_csv("infografica3.csv", sep=";")

In [ ]:
from bson.son import SON
import pprint
#start = datetime(2019, 1, 30, 0, 0, 0)
pipeline = [
        {"$match": {"categoria":"ES*"}},
        #{"$project": {"_id":1}},
         {"$group": {"_id":"$codice_treno", "ritardo_medio":{"$avg": "$ritardo_finale"},
                     "count": {"$sum": 1}}},
         {"$sort": SON([("ritardo_medio", -1), ("_id", -1)])}
]
pprint.pprint(list(db.trenitalia.aggregate(pipeline)))

In [74]:
from bson.son import SON
import pprint
#start = datetime(2019, 1, 30, 0, 0, 0)
pipeline = [
            {"$match": {"categoria":"ES*","origine":"TORINO P.NUOVA",
                    "destinazione":"VENEZIA SANTA LUCIA"}},
            #{"$project": {"fermate.nome_stazione":1,"tratta":1}},
        {"$group": {"_id":"$tratta", "count": {"$sum":1}}}
]
lista = (list(db.trenitalia.aggregate(pipeline)))

In [372]:
from bson.son import SON
import pprint
pipeline = [
        #{"$unwind": "$fermate"},
        {"$match": {"categoria": "ES*"}},
        {"$match": {"$or":[{ "stato": {"$ne":"parzialmente soppresso"}},
                           { "stato": {"$ne":"cancellato"}}]}},
        {"$project": {"data":0,"fermate.orario":0}}
        ]
lista = list(db.trenitalia.aggregate(pipeline))

for j in lista:
    j["_id"] = str(j["_id"])

with open('av_def.json', 'w') as fout:
    json.dump(lista , fout)

In [ ]:
pipeline = [
        {"$unwind": "$fermate"},
        {"$match": {"fermate.regione":"LOMBARDIA"}},
        {"$group": {"_id": "$fermate.nome_stazione", 
                    "ritardo_medio": {"$avg": "$fermate.ritardo"}}},
        {"$sort": SON([("ritardo_medio", -1)])}
]
pd.DataFrame(list(db.trenitalia.aggregate(pipeline))[:5])